<a href="https://colab.research.google.com/github/JeanMusenga/PhD-Thesis_2024_Musenga/blob/main/BERTSum_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/nlpyang/BertSum

https://web.stanford.edu/class/archive/cs/cs224n/cs224n.1214/reports/final_reports/report042.pdf

https://github.com/sudharsan13296/Getting-Started-with-Google-BERT/blob/main/6.%20Exploring%20BERTSUM%20for%20text%20summarization/6.07.%20Training%20the%20BERTSUM%20model%20.ipynb

In [1]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
import torch

In [3]:
file_path = './saved_file'
file_path = ('DataSampePilot.xlsx')

data = pd.read_excel(file_path)

In [2]:
# Function to summarize text using BERTSum
def summarize_with_bertsum(text, max_length=150, min_length=50):
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs, max_length=max_length, min_length=min_length, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [4]:
# Load your specific BERTSum model and tokenizer
# Replace these with your specific model paths or identifiers
tokenizer = BertTokenizer.from_pretrained('path_or_name_of_your_bertsum_tokenizer')
model = BertForSequenceClassification.from_pretrained('path_or_name_of_your_bertsum_model')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


OSError: path_or_name_of_your_bertsum_tokenizer is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
# Generate summaries for "Question_body" and "Answer_body"
data['Question_Summary'] = data['Question_body'].apply(summarize_with_bertsum)
data['Answer_Summary'] = data['Answer_body'].apply(summarize_with_bertsum)


In [ ]:
# Save the new dataframe with summaries
output_file_path = 'Summarized_Data.xlsx'
data.to_excel(output_file_path, index=False)


In [ ]:
# Display a few examples of the new dataframe
print(data[['Question_body', 'Question_Summary', 'Answer_body', 'Answer_Summary']].head())